The Poincaré inequality
===

$$
\| u \|_{L_2(\Omega)}^2 \leq c 
\left( \| \nabla u \|_{L_2(\Omega)}^2 + \left(\tfrac{1}{|\Omega|} \int_\Omega u \, dx \right)^2 \right)
$$

In [1]:
from ngsolve import *
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve.webgui import Draw 

mesh = MakeStructured2DMesh(nx=10,ny=10, mapping = lambda x,y : (x,y))
fes = H1(mesh, order=3)
u,v = fes.TnT()

L2Norm = BilinearForm(u*v*dx).Assemble()
H1SemiNorm = BilinearForm(grad(u)*grad(v)*dx).Assemble()
H1Norm = BilinearForm(grad(u)*grad(v)*dx+u*v*dx).Assemble()
pre = H1Norm.mat.Inverse()

vol = Integrate(1, mesh, VOL)
meanvalue = LinearForm(1/vol*v*dx).Assemble()
meanvalmat = BaseMatrix(meanvalue.vec)  # vector as N*1 matrix
M = H1SemiNorm.mat + meanvalmat @ meanvalmat.T

evals,evecs = solvers.PINVIT(M, L2Norm.mat, pre=pre, num=5, maxit=10, printrates=False)
print ("eigenvalues: ", list(evals))
print ("Norm of trace operator: ||tr||^2 = ", 1/evals[0])

eigenvalues:  [0.9999999999999937, 9.869604494897402, 9.869604494897622, 19.73920899176004, 39.487530570373124]
Norm of trace operator: ||tr||^2 =  1.0000000000000064


In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = evecs[0]
gfu.vec.data /= Integrate(gfu*dx, mesh) # normalize eigenfunction
Draw (gfu, mesh)